# Import libraries

In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact

# Import data

In [7]:
DATA_DIR = '/home/victor/Mestrado/Disciplinas/Visão Computacional/Projeto Final/bowfire/BowFire'
IMG_DIR = os.path.join(DATA_DIR, 'images')
MASK_DIR = os.path.join(DATA_DIR, 'masks')

In [8]:
list_imgs = [os.path.join(IMG_DIR, x) for x in os.listdir(IMG_DIR)]
list_masks = [os.path.join(MASK_DIR, x) for x in os.listdir(MASK_DIR)]

list_imgs = sorted(list_imgs)
list_masks = sorted(list_masks)

print('Total de imagens', len(list_imgs))
print('Total de máscaras', len(list_masks))

Total de imagens 119
Total de máscaras 119


In [9]:
@interact(count = (0, len(list_imgs)))
def view_images(count):
    global img
    global mask
    fig, ax = plt.subplots(1,2, figsize=(10,5))
    img = plt.imread(list_imgs[count])
    mask = plt.imread(list_masks[count])
    ax[0].imshow(img)
    ax[1].imshow(mask)
    
    for axs in ax:
        axs.axis('Off')

interactive(children=(IntSlider(value=59, description='count', max=119), Output()), _dom_classes=('widget-inte…

# Mahalanobis Distance


        Mahalonobis distance is the distance between a point and a distribution. 
        And not between two distinct points. 
        It is effectively a multivariate equivalent of the Euclidean distance.

https://www.machinelearningplus.com/statistics/mahalanobis-distance/

The formula to compute Mahalanobis distance is as follows:

$D^2 = (x-m)^{T} \cdot C^{-1} \cdot (x-m)$

**Where,**



$D^2$ is the square of the Mahalanobis distance. 

$x$          is the vector of the observation (row in a dataset), 

$m$          is the vector of mean values of independent variables (mean of each column), 

$C^{-1}$     is the inverse covariance matrix of independent variables. 

---

$(x – m)$ is essentially the distance of the vector from the mean. We then divide this by the covariance matrix (or multiply by the inverse of the covariance matrix).

If you think about it, this is essentially a multivariate equivalent of the regular standardization (z = (x – mu)/sigma). That is, z = (x vector) – (mean vector) / (covariance matrix).

In [39]:
def mahalanobis(img):
    '''
    Equation:
        (x-m)^T @ C^{-1} @ (x-m)
  
    Args:
        img = numpy array
    '''
    for c in img
    mah_dist = ( 
        ((x - np.mean(x)).T @ np.linalg.inv(np.cov(x))) @ (x - np.mean(x))
        ) 
    return mah_dist

SyntaxError: invalid syntax (<ipython-input-39-1189d3ba8737>, line 9)

In [20]:
import pandas as pd
filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/diamonds.csv'
df = pd.read_csv(filepath).iloc[:, [0,4,6]]
df.head()

,carat,depth,price
0,0.23,61.5,326
1,0.21,59.8,326
2,0.23,56.9,327
3,0.29,62.4,334
4,0.31,63.3,335


In [30]:
def mahalanobis(x, data):
    '''
    Args:
        img = numpy array
    '''
    mah_dist = ( 
        ((x - np.mean(data)) @ np.linalg.inv(np.cov(data.values))) @ (x - np.mean(data).T)
        ) 
    return mah_dist

In [32]:
X = df[['carat', 'depth', 'price']].head(50)
df_x['mahala'] = mahalanobis(x=X, data=df[['carat', 'depth', 'price']])

MemoryError: Unable to allocate 21.7 GiB for an array with shape (53940, 53940) and data type float64

interactive(children=(IntSlider(value=59, description='count', max=119), Output()), _dom_classes=('widget-inte…

In [45]:
def mahalanobis(x=None, data=None, cov=None):
    # (x-m)^T @ C^{-1} @ (x-m)
    """Compute the Mahalanobis Distance between each row of x and the data  
    x    : vector or matrix of data with, say, p columns.
    data : ndarray of the distribution from which Mahalanobis distance of each observation of x is to be computed.
    cov  : covariance matrix (p x p) of the distribution. If None, will be computed from data.
    """
    m = np.mean(data)

    mah = np.dot((x - m), np.linalg.inv(np.cov(data.values))) @ (x - m).T
    return mah

In [46]:
df_x = df[['carat', 'depth', 'price']].head(500)
df_x['mahala'] = mahalanobis(x=df_x, data=df[['carat', 'depth', 'price']])
df_x.head()

ValueError: Wrong number of items passed 500, placement implies 1